In [19]:
%load_ext autoreload
%autoreload 2

import os
import random
import sys

sys.path.append("../")

from matplotlib import pyplot as plt
import featuretools as ft
from woodwork.logical_types import Categorical, Double, Datetime, Age
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from src.utils import DEFAULT_RANDOM_STATE, set_random_state

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
set_random_state(state=DEFAULT_RANDOM_STATE)
sns.set_theme()

class Paths:
    data_dir = "../data/src/"
    submit_dir  = "../data/submissions/"
    out_dir = "../data/stepik_out/"
    
paths = Paths()

regenerate = True
mode = "train"  # "train" (only train dataset), "sub" (no y_test)

In [3]:
car_train = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_train.csv")
rides_info = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/rides_info.csv")
driver_info = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/driver_info.csv")
fix_info = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/fix_info.csv")

In [67]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/825511/meta_feautres_data.csv")
df = df.pivot_table(values="target", index="index", columns="model").reset_index()
names = ["catboost", "lgbm", "xgb"]
operations = ["div", "dif"]
for lname in names:
    for rname in names:
        for op in operations:
            if op == "div":
                col = df[lname] / df[rname]
            else:
                col = df[lname] - df[rname]
            df[f"{lname}_{op}_{rname}"] = col
for name in names:
    df.rename({name: "target_" + name}, axis=1, inplace=True)
df

model,index,target_catboost,target_lgbm,target_xgb,catboost_div_catboost,catboost_dif_catboost,catboost_div_lgbm,catboost_dif_lgbm,catboost_div_xgb,catboost_dif_xgb,...,lgbm_div_lgbm,lgbm_dif_lgbm,lgbm_div_xgb,lgbm_dif_xgb,xgb_div_catboost,xgb_dif_catboost,xgb_div_lgbm,xgb_dif_lgbm,xgb_div_xgb,xgb_dif_xgb
0,0,65.927922,49.622930,55.741352,1.0,0.0,1.328578,16.304993,1.182747,10.186570,...,1.0,0.0,0.890235,-6.118422,0.845489,-10.186570,1.123298,6.118422,1.0,0.0
1,1,44.248116,43.244205,42.661655,1.0,0.0,1.023215,1.003911,1.037187,1.586461,...,1.0,0.0,1.013655,0.582550,0.964146,-1.586461,0.986529,-0.582550,1.0,0.0
2,2,43.807966,45.389293,42.217470,1.0,0.0,0.965161,-1.581327,1.037674,1.590496,...,1.0,0.0,1.075131,3.171823,0.963694,-1.590496,0.930120,-3.171823,1.0,0.0
3,3,45.661121,44.769051,40.969734,1.0,0.0,1.019926,0.892070,1.114509,4.691387,...,1.0,0.0,1.092735,3.799317,0.897256,-4.691387,0.915135,-3.799317,1.0,0.0
4,4,43.653036,40.462658,38.243040,1.0,0.0,1.078847,3.190378,1.141464,5.409996,...,1.0,0.0,1.058040,2.219618,0.876068,-5.409996,0.945144,-2.219618,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,580,43.372847,44.205443,39.795555,1.0,0.0,0.981165,-0.832596,1.089892,3.577292,...,1.0,0.0,1.110814,4.409888,0.917522,-3.577292,0.900241,-4.409888,1.0,0.0
581,581,44.168977,42.959736,42.351254,1.0,0.0,1.028148,1.209241,1.042920,1.817723,...,1.0,0.0,1.014368,0.608482,0.958846,-1.817723,0.985836,-0.608482,1.0,0.0
582,582,44.393186,43.844842,44.074200,1.0,0.0,1.012506,0.548344,1.007237,0.318986,...,1.0,0.0,0.994796,-0.229358,0.992815,-0.318986,1.005231,0.229358,1.0,0.0
583,583,43.649487,46.200331,40.630420,1.0,0.0,0.944787,-2.550844,1.074306,3.019067,...,1.0,0.0,1.137087,5.569911,0.930834,-3.019067,0.879440,-5.569911,1.0,0.0


In [63]:
df.to_csv("../data/stepik_out/chapter_6_2_5.csv", index=False)